# Recurrent Neural Networks

In [1]:
import torch
import os
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm

In [2]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
    
    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __len__(self):
        return len(self.word2idx)

In [3]:
class TextProcess(object):
    def __init__(self):
        self.dictionary = Dictionary()
        
    def get_data(self, path, batch_size=20):
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)
        # 1D tensor that contains index of all the  words in the file
        rep_tensor = torch.LongTensor(tokens)
        index = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    rep_tensor[index] = self.dictionary.word2idx[word]
                    index += 1
        num_batches = rep_tensor.shape[0] // batch_size
        # Remove remainder
        rep_tensor = rep_tensor[:num_batches * batch_size]
        return rep_tensor.view(batch_size, -1)

In [4]:
embedding_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 20
batch_size = 20
timesteps = 30
learning_rate = 0.002

In [5]:
corpus = TextProcess()

In [6]:
rep_tensor = corpus.get_data('data/alice.txt', batch_size).to('cuda')

In [7]:
print(rep_tensor.shape)

torch.Size([20, 1484])


In [8]:
vocab_size = len(corpus.dictionary)
print(vocab_size)

5290


In [9]:
num_batches = rep_tensor.shape[1] // timesteps
print(num_batches)

49


In [10]:
class TextGenerator(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, x, states):
        # Word embedding
        x = self.embed(x)
        # Reshape the input tensor
        # x = x.view(batch_size,timesteps,embed_size)
        out, (h, c) = self.lstm(x, states)
        # Reshape the output from (samples, timesteps, output_features) to a shape for FC
        # (batch_size * timesteps, hidden_size)
        out = out.reshape(out.size(0) * out.size(1), out.size(2))
        out = self.linear(out)
        return out, (h, c)

In [11]:
model = TextGenerator(vocab_size, embedding_size, hidden_size, num_layers).to('cuda')

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
for epoch in range(num_epochs):
    # hidden and cell states
    states = (
        torch.zeros(num_layers, batch_size, hidden_size).to('cuda'),
        torch.zeros(num_layers, batch_size, hidden_size).to('cuda')
    )
    for i in range(0, rep_tensor.size(1) - timesteps, timesteps):
        # Get mini batch and targets
        inputs = rep_tensor[:, i:i+timesteps].to('cuda')
        targets = rep_tensor[:, (i+1):(i+timesteps+1)].to('cuda')
        outputs, _ = model(inputs, states)
        loss = loss_fn(outputs, targets.reshape(-1))
        
        model.zero_grad()
        loss.backward()
        
        clip_grad_norm(model.parameters(), 0.5)
        optimizer.step()
        
        step = (i + 1) // timesteps
        if step % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item():.4f}')

C:\Users\msoum\AppData\Local\Temp\ipykernel_7404\525320973.py:17: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  clip_grad_norm(model.parameters(), 0.5)


Epoch [1/20] Loss: 8.5715
Epoch [2/20] Loss: 5.9308
Epoch [3/20] Loss: 5.2171
Epoch [4/20] Loss: 4.6095
Epoch [5/20] Loss: 4.1298
Epoch [6/20] Loss: 3.7112
Epoch [7/20] Loss: 3.2626
Epoch [8/20] Loss: 2.7714
Epoch [9/20] Loss: 2.4098
Epoch [10/20] Loss: 2.0225
Epoch [11/20] Loss: 1.6661
Epoch [12/20] Loss: 1.3753
Epoch [13/20] Loss: 1.1592
Epoch [14/20] Loss: 0.8643
Epoch [15/20] Loss: 0.6199
Epoch [16/20] Loss: 0.3834
Epoch [17/20] Loss: 0.2883
Epoch [18/20] Loss: 0.1568
Epoch [19/20] Loss: 0.1019
Epoch [20/20] Loss: 0.0798


In [17]:
with torch.no_grad():
    with open('data/results.txt', 'w') as f:
        states = (
            torch.zeros(num_layers, 1, hidden_size).to('cuda'),
            torch.zeros(num_layers, 1, hidden_size).to('cuda')
        )
        inp = torch.randint(0, vocab_size, (1,)).long().unsqueeze(1).to('cuda')
        
        for i in range(500):
            output, _ = model(inp, states)
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()
            inp.fill_(word_id)
            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)